<a href="https://colab.research.google.com/github/hridog00/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Neuroevolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
    from keras import backend as K
    K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.datasets import mnist
from keras.utils import to_categorical


Using TensorFlow backend.


In [0]:
#Load the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train / 255) - 0.5
x_test = (x_test / 255) - 0.5
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
train_images = np.expand_dims(x_train, axis=3)
test_images = np.expand_dims(x_test, axis=3)

x_train = x_train.reshape(-1,28, 28, 1)  
x_test = x_test.reshape(-1,28, 28, 1)
input_shape = (28, 28, 1)

In [0]:
#params of the algorithm
N = 200
G = 20
n_conv_layer = 0
n_dense_layer = 2
pc = 0.7
pm = 0.2
elitism = 0.1
evaluated_models = {}

In [0]:
def create_indiv():
    n_c = random.randint(0,n_conv_layer+1)
    n_d = random.randint(0,n_dense_layer+1)
    #conv parameters
    kernel = (3,5) 
    filters = (16,32)
    stride = (1,)
    pooling = (2,)
    function = ('relu',)
    #dense parameters
    neurons = (32,64)
    dropout = (0.4,)
    function_d = ('sigmoid', 'tanh', 'relu')

    indiv = []
    for i in range (n_c):
        conv_layer = []
        conv_layer.append('C') #0
        conv_layer.append(random.choice(function)) #1
        conv_layer.append(random.choice(filters))  #2
        conv_layer.append(random.choice(kernel))   #3
        conv_layer.append(random.choice(stride))   #4
        conv_layer.append(random.choice(pooling))  #5
        indiv.append(conv_layer)
    for i in range (n_d):
        dense_layer = []
        dense_layer.append(random.choice('D'))
        dense_layer.append(random.choice(function_d))
        dense_layer.append(random.choice(neurons))
        dense_layer.append(random.choice(dropout))
        indiv.append(dense_layer)
    return indiv

In [0]:
def get_n_layers(layers):
    n_c = 0
    n_d = 0
    for l in layers:
        if l[0] == 'C':
            n_c += 1
        else:
            n_d += 1
    return n_c, n_d

In [0]:
def build_model(layers):
    n_c, _ = get_n_layers(layers)
    global input_shape
    output_shape = 10
    model = Sequential()
    input_layer = 0
    conv = 0
    for i in layers:
        if conv == n_c and n_c != 0:
            print('Flatten later added')
            model.add(Flatten())
            conv +=1
        if  input_layer == 0 :
            if i[0] == 'C':
                model.add(Conv2D(i[2], kernel_size=i[3], strides=i[4], activation=i[1], input_shape=input_shape))
                model.add(MaxPooling2D(pool_size=i[5]))
                conv += 1
            else:
                if len(input_shape) == 3:
                    input_shape = input_shape[0] * input_shape[1]
                model.add(Dense(i[2], input_dim=input_shape[0], activation=i[1]))
                model.add(Dropout(i[3]))
            input_layer = 1
        else:
            if i[0] == 'C':
                model.add(Conv2D(i[2], kernel_size=i[3], strides=i[4], activation=i[1]))
                model.add(MaxPooling2D(pool_size=i[5]))
                conv += 1
            else:
                model.add(Dense(i[2], activation=i[1]))
                model.add(Dropout(i[3]))
    model.add(Dense(output_shape, activation='softmax'))
    return model

In [0]:
def create_generation(N):
    G = [create_indiv() for _ in range(N)]
    return G

In [0]:
def evaluate(indiv):
    global evaluated_models
    global x_train
    global x_test
    global y_test
    global y_train
    if str(indiv) not in evaluated_models.keys():
        n_c, _ = get_n_layers(indiv)
        if n_c == 0:
            x_train = x_train.reshape(x_train.shape[0], -1)
            x_test = x_test.reshape(x_test.shape[0], -1)
            m = build_model(indiv)
            m.summary()
            print('Method created')
            m.compile('adam',loss='categorical_crossentropy', metrics=['accuracy'],)
            m.fit(np.asarray(x_train),np.asarray(y_train), epochs =50, verbose=True,  batch_size=64,  callbacks= [EarlyStopping(monitor='loss', patience=5, verbose=1)])
        try:
            
            
            
            acc = m.evaluate(x_test,y_test,verbose=False)[1]
            evaluated_models[str(indiv)] = acc
            
        except:
           acc = 0
        return acc
    else:
        return evaluated_models[str(indiv)]

In [0]:
#indiv = create_indiv()
#print(indiv)
accuracy = evaluate(indiv)
accuracy

0

In [0]:
m = build_model(indiv)
m.compile('adam',loss='categorical_crossentropy', metrics=['accuracy'],)
m.fit(np.asarray(x_train),np.asarray(y_train), epochs =50, verbose=True,  batch_size=64,  callbacks= [EarlyStopping(monitor='loss', patience=5, verbose=1)])

Flatten later added
Epoch 1/50
27584/60000 [============>.................] - ETA: 4s - loss: 0.4942 - acc: 0.8482

KeyboardInterrupt: ignored

In [0]:
evaluated_models

{"[['C', 'relu', 16, 3, 1, 2], ['C', 'relu', 16, 3, 1, 2], ['D', 'sigmoid', 64, 0.4], ['D', 'tanh', 64, 0.4]]": 0.9905,
 "[['C', 'relu', 32, 3, 1, 2], ['D', 'relu', 32, 0.4]]": 0.9841}

In [0]:
a = np.asarray([[[1,1,1],[1,1,1],[1,1,1]], [[3,3,3],[3,3,3],[3,3,3]]])

In [0]:
a.reshape(a.shape[0], -1)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [3, 3, 3, 3, 3, 3, 3, 3, 3]])